In [1]:
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

path = os.path.realpath(os.path.join(os.getcwd(),'../'))
sys.path.insert(0, path)
import astro_ts_datasets
import medical_ts_datasets

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
tf.compat.v1.enable_eager_execution()

In [4]:
as_supervised = True
split = 100
split = 'train'
#split = 'train[:{0}%]+test[:{1}%]'.format(split, split)
#split = 'train[10:12]'

In [5]:
drop_fraction = 0.5

In [6]:
def augment(*data):
    if as_supervised:
        inputs, label = data
        static, times, values, measurements, lengths = inputs[0:5]
        condition = tf.random.uniform(tf.shape(measurements)) < \
                    tf.ones(tf.shape(measurements), dtype=tf.float32) * tf.random.uniform(shape=(1,), minval=0, maxval=drop_fraction)
        measurements = tf.where(
            condition,
            tf.cast(tf.zeros(tf.shape(measurements)), dtype=tf.bool),
            measurements
        )
        return (static, times, values, measurements) + inputs[4:], label
    else:
        return data

In [7]:
def preprocess(*data):
    if as_supervised:
        inputs, label = data
        static, times, values, measurements, lengths = inputs[0:5]
        times = times - times[0]
        return (static, times, values, measurements) + inputs[4:], label
    else:
        return data

In [ ]:
ds, info = tfds.load(name='plasticc', split=split, with_info=True, as_supervised=as_supervised)
ds = ds.map(augment)
ds = ds.map(preprocess)

In [ ]:
info

In [ ]:
i = 0 
for d in ds.take(200000):
    i += 1
    inputs, label = d
    #static, times, values, measurements, lengths, static_errors, value_errors = inputs
    #plt.figure(figsize=(8,4))
    #plt.imshow(values.numpy().T)
    #plt.show()
    #plt.figure(figsize=(8,4))
    #plt.imshow(measurements.numpy().T)
    #plt.show()

In [ ]:
i